In [ ]:
!pip install deap

In [ ]:
import random, time, turtle, math
import numpy as np
from deap import base, creator, tools
from IPython import display as dis
import matplotlib.pyplot as plt


In [ ]:
class agent():
    
    def __init__(self,group,wealth):
        self.group = group
        self.wealth = wealth
    
    

Above I have defined a simple agent. This agent only needs to be a simple object that holds the values of its group and its wealth in order for my program to work. N number of these agents will be created which are then split evenly into the four societies and have a starting wealth of zero. These agents are all given their own individual neural networks which form the basis of my program and determine the genotypes and phenotypes. 

The genotypes are represented by the number of neurons in each agent's neural network. Each neural network has the same shape and thus each agent’s genotype is the same. The shape of the neural network is that of 3 input neurons, 6 hidden neurons and 4 output neurons. The three input neurons are needed to take in information about games played. They take in whether or not the agent "won" in a game, whether or not the agent was in the same group as its opponent and whether or not the opponent had more wealth than the agent. There needed to be 4 output neurons to represent the four societies the agent could choose to switch to.
The number of hidden layer neurons was chosen through research online. From the research i determined that the number of hidden neurons should equal 2/3 of the input nodes plus the output nodes which is of course 6. Through testing having 6 neurons seemed to be ideal to get the results I wanted and to avoid overfitting. So, through all of this I ended up with a 13 number genotype.

The phenotypes are the weights of the individual neurons in each agent's neural network. The weights of the neurons ultimately decide if a given agent should swap groups or not. These weights are initially given weights randomly between -1 and 1. Mutation was used in order to evolve the agents and give variance to these weights. This was done by using a gaussian distribution which had a mean of 0 allowing values to be selected from either side of 0. Using a sigma value of 0.2 allows for mutation to be selected within the range of -1 and 1. The independent mutation probability is selected at 0.2. I also employed tournament selection to reduce stochastic noise. It is used to add variance wow still allowing the fittest individuals to pass on. I found using a tournament size of 10 was appropriate as it showed a nice balance of variance and allowing the fittest individuals to pass through. 


In [ ]:
class NN(object):
    def __init__(self, numInput, numHidden1,numOutput):
        self.fitness = 0,
        self.numInput = numInput + 1 # Add bias node from input to hidden layer 1 only,
        self.numHidden1 = numHidden1 # Feel free to adapt the code to add more biases if you wish,
        self.numOutput = numOutput

        self.w_i_h1 = np.random.randn(self.numHidden1, self.numInput) 
        self.w_h1_o = np.random.randn(self.numOutput, self.numHidden1)

        self.ReLU = lambda x : max(0,x)

    def sigmoid(self,x):
        try:
            ans = (1 / (1 + math.exp(-x)))
        except OverflowError:
            ans = float('inf')
        return ans

In [ ]:
class NN(NN):
        def feedForward(self, inputs):
            inputsBias = inputs[:]
            inputsBias.insert(len(inputs),1)             # Add bias input
    
            h1 = np.dot(self.w_i_h1, inputsBias)         # feed input to hidden layer 1
            h1 = [self.ReLU(x) for x in h1]              # Activate hidden layer1
              
            output = np.dot(self.w_h1_o, h1)             # feed to output layer
            output = [self.sigmoid(x) for x in output]   # Activate output layer
            return output

In [ ]:
class NN(NN):
        
        def getWeightsLinear(self):
            flat_w_i_h1 = list(self.w_i_h1.flatten())
            flat_w_h2_o = list(self.w_h1_o.flatten())
            return( flat_w_i_h1 + flat_w_h1_o )
    
        def setWeightsLinear(self, Wgenome):
            numWeights_I_H1 = self.numHidden1 * self.numInput
            numWeights_H2_O = self.numOutput * self.numHidden1
    
            self.w_i_h1 = np.array(Wgenome[:numWeights_I_H1])
            self.w_i_h1 = self.w_i_h1.reshape((self.numHidden1, self.numInput))
            
    
            self.w_h1o = np.array(Wgenome[(numWeights_I_H1):])
            self.w_h1o = self.w_h1o.reshape((self.numOutput, self.numHidden1))

In [ ]:
num_agents = 400
wealth = 0
num_games = 3200




tournamentSize = 10
muProb = 0.2
population = 100
generations = 200
muSigma = 0.2


inputLayer = 3
HiddenNodes = 6
outputLayer = 4

IND_SIZE = ((inputLayer+1) * (HiddenNodes)) + ((HiddenNodes) * (outputLayer))

myNet1 = NN(inputLayer, HiddenNodes, outputLayer)
myNet2 = NN(inputLayer, HiddenNodes, outputLayer)


In [ ]:
def run_game(pop,num_games,adaptive):
    count = 0
    popDict = {}
    for popIndex in range(len(pop)):
        #need to add a check that no two sums are the same
        #if any are, add a small number to one of the values in the indiv
        
        summed = round(sum(pop[popIndex]),5)
        
        if summed in popDict:
            summed += summed/1000
            summed = round(summed,5)
            
        popDict[summed] = agent(popIndex%4,0)
        
        
    hm = np.zeros(len(pop))
    for i in range(num_games):

        player1Index = random.randint(0,len(pop)-1)
        player2Index = random.randint(0,len(pop)-1)
        while player1Index == player2Index:
            player2Index = random.randint(0,len(pop)-1)

        players = [popDict[round(sum(pop[player1Index]),5)],popDict[round(sum(pop[player2Index]),5)]]
        
    
        
        myNet1.setWeightsLinear(pop[player1Index])
        myNet2.setWeightsLinear(pop[player2Index])

        decision = []

        for j in range(len(players)):
            if players[j].group == 0:
                decision.append(0)
            elif players[j].group == 1:
                if players[(j+1)%2].group == 1:
                        decision.append(0)
                else:
                    decision.append(1)
            elif players[j].group == 2:
                if players[(j+1)%2].group == 2:
                    decision.append(1)
                else:
                    decision.append(0)
            
            else:
                decision.append(1)
                


        
        won = []
        temp1 = players[0].wealth
        temp2 = players[1].wealth
        if (decision[0] == 0) and (decision[1] == 0):
            players[0].wealth += 4
            players[1].wealth += 4
            won.append(1)
            won.append(1)
        elif (decision[0] == 1) and (decision[1] == 0):
            players[0].wealth += 6
            won.append(1)
            won.append(0)
        elif (decision[0] == 0) and (decision[1] == 1):
            players[1].wealth += 6
            won.append(0)
            won.append(1)
        else:
            players[0].wealth += 1
            players[1].wealth += 1
            won.append(0)
            won.append(0)
            
        if(players[0].wealth - temp1 == 8):
            players[0].wealth -= 4
            players[1].wealth -= 4
            
        if(players[0].wealth - temp1 == 2):
            players[0].wealth -= 1
            players[1].wealth -= 1
        
        

        nets = [myNet1, myNet2]
        for j in range(2):
            
            same_group = int(players[j].group == players[(j+1)%2].group)
            wealth_comparison = int(players[j].wealth >= players[(j+1)%2].wealth)
            winner = won[j]
            
            state = [same_group,winner,wealth_comparison]

            output = nets[j].feedForward(state)
            new_group = np.argmax(output, axis=0)
            #print(output)
            #print(new_group)
            if adaptive == True:
                players[j].group = new_group
    
    
    grouplist = []
    for i in range(len(pop)):
        grouplist.append(popDict[round(sum(pop[i]),5)].group)
    
    
    
    return popDict, grouplist

Above you can see the code which is responsible for actually running the games and determining the fitness. Here the weights of each agents’ neurons are taken in to be changed based on the games. Firstly, two of these agents are randomly selected to play the game together. Depending on their society and how this society interacts with its opponent’s society wealth is added on accordingly. Then for each agent it uses the neural network to determine which society it should join. Three inputs are used to be taken in by the neural network which are whether the agent "won" the game, whether or not the agent was in the same group as its opponent and whether or not the opponent had more wealth than the agent. The outputs of this neural network refer to the four societies which then the agent can join. The output neuron which results in the highest value will be the society which the agent joins. 

This process is repeated for a user selected number of times each time picking new random agents to play against each other. The fitness which I have used to evolve my agents is that of wealth. Agents which have a higher fitness after the selected number of games will be more likely to proceed in the mutation process. The code above returns the fitness of every agent that played in the games to be assessed and evolved.

This code is also used to run the game in a non-adaptive setting. This is simply done by making it so the agents can’t change their society regardless of the outcomes of the neural networks.

    

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -1.0, 1.0)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)



In [ ]:

def evaluate(indiv, popDict):
    key = round(sum(indiv), 5)
    if key in popDict:
        return popDict[round(sum(indiv), 5)].wealth,
    return 0,



In [ ]:
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selTournament, tournsize=tournamentSize)

toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=muSigma, indpb=muProb)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)



In [ ]:
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [ ]:
logbook = tools.Logbook()

In [ ]:

def evolve(adaptive):
    data = []
    meanlist=[]
    maxlist=[]
    minlist=[]
    grouplist=[0,0,0,0]
    grouplist2 = [[],[],[],[]]
    total_score =0
    pop = toolbox.population(n=num_agents)
    for g in range(generations):
        print("-- Generation %i --" % g)
        #myNet.setWeightsLinear(indiv)

        popDict = run_game(pop,num_games,adaptive)

        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        for mutant in offspring:
            toolbox.mutate(mutant)
            del mutant.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]

        popDict,groups = run_game(invalid_ind,num_games,adaptive)
        fitnesses = [toolbox.evaluate(indiv, popDict) for indiv in invalid_ind]
        mean = 0
        count = 0
        for i in fitnesses:
            mean += i[0]

        mean = mean/len(fitnesses)

        print("average wealth: ",mean)


        for i in groups:
            if i == 0:
                grouplist[0] += 1
            elif i == 1:
                grouplist[1] += 1
            elif i == 2:
                grouplist[2] += 1
            elif i == 3:
                grouplist[3] += 1

        for i in range(len(grouplist)):
            grouplist[i] = grouplist[i]/len(groups)
            grouplist2[i].append(round(grouplist[i],4))

        meanlist.append(mean)
        maxlist.append(max(fitnesses))
        minlist.append(min(fitnesses))

        for ind, fit in zip(invalid_ind, fitnesses):
             ind.fitness.values = fit
        pop[:] = offspring
        record = stats.compile(pop)
        logbook.record(gen=g, **record)
        data.append(fitnesses)
        
    return meanlist,maxlist,minlist,data,grouplist2

Above is the code which handles the evolution of the agents. It is responsible for setting up the agents and passing them of into the games to have their fitnesses determined. It then takes the fitnesses and decides which agents to mutate and which agents should be allowed to progress in the tournament. This is done for a user selected number of generations. Even though the agents start of with 0 wealth each game and in their correct groups the neural network information is kept and used in the next run of games. This allows each progressive generation to learn faster and reach higher fitnesses.

In [ ]:
start = time.time()
meanlist,maxlist,minlist,data,grouplist = evolve(True)
finish = time.time()

print("Time taken for adaptive program: ", finish - start)

In [ ]:
plt.ion()


x = range(0,len(meanlist))        
y = meanlist
plt.title("Wealth per generation") 
plt.xlabel("Generation") 
plt.ylabel("Wealth") 
plt.plot(x,y)
plt.show()


plt.title("Wealth per generation") 
plt.xlabel("Generation") 
plt.ylabel("Wealth") 
plt.plot(x,maxlist,label = "Maximum")
plt.plot(x,minlist,label = "Minimum")
plt.legend(loc='best')
plt.show()



plt.title("Group percentage per generation") 
plt.xlabel("Generation") 
plt.ylabel("Percentage") 
plt.plot(x,grouplist[0],label = "Saints")
plt.plot(x,grouplist[1],label = "Buddies")
plt.plot(x,grouplist[2],label = "Fight Club")
plt.plot(x,grouplist[3],label = "Vandals")
plt.legend(loc='best')
plt.show()





Above can be seen the results of running the games with adaptation turned on.

The average wealth against generations is plotted in the first graph. Here we can see an initial dip in the wealth as the program assesses the viability of different strategies and then a sharp uptick as it finds its strategy. This then levels of around the level of 8(number of games / number of agents) which i determine to be the theoretical maximum average wealth of this game. This shows that the program is adapting and finding the best solution to this task.

Next can be seen the maximum and minimum wealth per generation. The minimum fluctuates around the same values fairly consistently for the duration. The maximum initially increases before then also fluctuating around similar values. This shows that variance is still in play as we minimum scores are still consistent but that the program is clearly adapting as maximum scores increase.

The next graph reveals the strategy which the program is ultimately settling on. This graph shows the percentage of agents in each of the four societies against generations. It clearly shows that the preferred strategy employed by the adaptive program is to favour the "buddies" society. Over time the other societies sharply fall approaching 0 agents in these societies while all agents move to the buddies society. This makes sense as cooperating gives them the most consistent high rewards and by being in the buddies group they can also make sure they never get zero wealth rewards. This is because unless it’s against its own group it will always not cooperate netting itself at least a point and sometimes six. It also makes sense that all the agents would want to be in buddies seeing as this way they can all cooperate together with low risk and maximize their individual wealth.


In [ ]:
start = time.time()
meanlist2,maxlist2,minlist2,data2,grouplist2 = evolve(False)
finish = time.time()
print("Time taken for non-adaptive program: ", finish - start)


In [ ]:
plt.ion()
print(len(grouplist2))

x = range(0,len(meanlist))        
y = meanlist2
plt.title("Wealth per generation") 
plt.xlabel("Generation") 
plt.ylabel("Wealth") 
plt.plot(x,y)
plt.show()


plt.title("Wealth per generation") 
plt.xlabel("Generation") 
plt.ylabel("Wealth") 
plt.plot(x,maxlist2,label = "Maximum")
plt.plot(x,minlist2,label = "Minimum")
plt.legend(loc='best')
plt.show()





Above can be seen the results of running the games with adaptation turned off.

The first graph shows the average wealth plotted against generation. Obviously there is no trend in the graph as there is no learning going on and each generation is going to start from the same point and average around the same. As a result, the average wealth per generation can be seen to be much lower than the adaptive program.

This lack of learning can also be shown in the second graph which shows the maximum and minimum wealth per generation. Here we see the same with no trend in either plot showing that the program just stagnates after the first generation and isn’t learning.


The time taken for each program is displayed below each of their respective results, the program was run on google colab with no hardware acceleration.

In [ ]:
from scipy import stats


format_data = []
format_data2 = []
for i in data[-1]:
    format_data.append(i[0])
format_data = np.array(format_data)

for i in data2[-1]:
    format_data2.append(i[0])
format_data2 = np.array(format_data2)





KSStatistic, pvalue = stats.epps_singleton_2samp(format_data2, format_data)
print("Epps-Singleton test:")
print("P-value: "+ str(pvalue))

statistic ,critical_values, significance_level = stats.anderson_ksamp([format_data2, format_data])
print("Anderson-Darling test :")
print("Significance level: "+str(significance_level))

KSStatistic, pvalue = stats.ks_2samp(format_data2, format_data,alternative='greater', mode='auto')
print("Two-sample Kolmogorov-Smirnov test:")
print("P-value: "+ str(pvalue))
print("\n")


print("Average adaptive wealth: "+ str(sum(format_data)/len(format_data)))
print("Standard diviation of adaptive wealth: "+ str(np.std(format_data)))
print("\n")

print("Average non-adaptive wealth: "+ str(sum(format_data2)/len(format_data2)))
print("Standard diviation of non-adaptive wealth: "+ str(np.std(format_data2)))



In the above code I further compared the adaptive and no adaptive programs through a series of statistical tests. In these tests I compared the game of each programs last generation. To do this it must be determined that the adapted wealth is of a different distribution to the non-adapted wealth.

I did this by using the Epps-Singleton test, the Anderson-Darling test and the Twi-sample Kolmogorov-Smirnov test. Using a threshold of 0.01 it can be clearly seen that they are of different distributions as the null hypothesis rejects due to the fact that they all fall far below the threshold. This shows that there is a definite behaviour which the adapted program is exhibiting meaning it is definitely adapting to its input.

However, these tests do not tell us if this behaviour is necessarily positive. To statistically determine this fact I calculated the average and standard deviation of each of the programs. A higher mean with lower standard deviation would indicate positive change which is what we ultimately see. This shows that higher wealth across the population was achieved and proves the program learned positive change


